In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn import metrics
import xgboost as xgb

# Creating the dataset used by the SNGuess boosted decision tree

We start from where we left it last time and load the file with features `snguess_training.csv` that we created in the notebook `snguess_training_data.ipynb`. 

In [2]:
features = pd.read_csv('snguess_training.csv')
features

,cut_pp,jd_det,jd_last,ndet,mag_det,mag_last,t_lc,rb_med,drb_med,distnr_med,...,snname,slope_fall_g,slope_fall_r,rcf_class,rcf_sn,rcf_agn,rcf_cv,targeted_by_bts,subclass,snclass
0,0,2.458434e+06,2.458434e+06,1,17.577295,17.577295,0.000000,0.433333,NaN,1.078758,...,ZTF18aaxgaxw,NaN,NaN,NaN,False,False,False,False,NaN,NaN
1,0,2.458434e+06,2.458434e+06,1,17.577295,17.577295,0.000000,0.433333,NaN,1.078758,...,ZTF18aaxgaxw,NaN,NaN,NaN,False,False,False,False,NaN,NaN
2,0,2.458434e+06,2.458434e+06,1,17.577295,17.577295,0.000000,0.433333,NaN,1.078758,...,ZTF18aaxgaxw,NaN,NaN,NaN,False,False,False,False,NaN,NaN
3,0,2.458434e+06,2.458434e+06,1,17.577295,17.577295,0.000000,0.433333,NaN,1.078758,...,ZTF18aaxgaxw,NaN,NaN,NaN,False,False,False,False,NaN,NaN
4,0,2.458434e+06,2.458434e+06,1,17.577295,17.577295,0.000000,0.433333,NaN,1.078758,...,ZTF18aaxgaxw,NaN,NaN,NaN,False,False,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250320,0,2.458663e+06,2.458668e+06,2,17.056499,17.605425,4.979271,0.649286,0.214382,0.762824,...,ZTF19abddmnz,0.110242,NaN,NaN,False,False,False,False,NaN,NaN
250321,0,2.458668e+06,2.458671e+06,3,19.795700,19.604424,2.981759,0.897143,0.999895,1.378160,...,ZTF19abddoln,NaN,NaN,NaN,False,False,False,False,NaN,NaN
250322,0,2.458668e+06,2.458668e+06,2,19.795700,19.865593,0.020556,0.903571,0.999870,1.513054,...,ZTF19abddoln,NaN,NaN,NaN,False,False,False,False,NaN,NaN
250323,0,2.458668e+06,2.458674e+06,4,19.795700,19.511892,5.982535,0.903571,0.999920,1.470891,...,ZTF19abddoln,NaN,NaN,NaN,False,False,False,False,NaN,NaN


## Marking short-lived transients
We will next mark transients for which we never got much data (less than six final detections). The reason is that these are often real SNe but for example where the survey did not continue following the field. RCF would not have tried to type these. Note that this will also include many potential interesting, short-lived candidates. The flag `rcf_gone` is added to reflect this.

In [3]:
rcf_gone = pd.Series(False, index=features[features.ndet > 6].snname.unique(), name="rcf_gone")
features = features.merge(rcf_gone, how='left', left_on='snname', right_index=True).fillna(value={'rcf_gone':True})

In [4]:
print('We found %s persistent transisnts (scaled from nbr_det prm)' % features.rcf_gone.value_counts()[False])
print('We found %s non persistent transisnts (scaled from nbr_det prm)' % features.rcf_gone.value_counts()[True])

We found 178035 persistent transisnts (scaled from nbr_det prm)
We found 72290 non persistent transisnts (scaled from nbr_det prm)


## Marking classes

In [5]:
print("Number of unique sne: %s"%(len(np.unique(features['snname']))))
print("Number of RCF sne: %s"%(len(np.unique(features[ features['rcf_sn'] ]['snname']))))
print("Number of agn: %s"%(len(np.unique(features[ features['rcf_agn'] ]['snname']))))
print("Number of cv: %s"%(len(np.unique(features[ features['rcf_cv'] ]['snname']))))
print("Number of gone: %s"%(len(np.unique(features[ features['rcf_gone'] ]['snname']))))

Number of unique sne: 45397
Number of RCF sne: 1601
Number of agn: 174
Number of cv: 119
Number of gone: 34298


In [6]:
features.loc[:,'class'] = 0
features.loc[features['rcf_gone'],'class'] = 1
features.loc[features['rcf_cv'],'class'] = 2
features.loc[features['rcf_agn'],'class'] = 3
features.loc[features['rcf_sn'],'class'] = 4

In [7]:
print('Number of alerts:', len(features[ features['class']>0 ]))
print('Number of candidates:', len(features[ features['class']>0 ].snname.unique()))

Number of alerts: 109599
Number of candidates: 35899


In [8]:
features.to_csv('snguess_features_class.csv', index_label=False)

## Data cuts
SNGuess (v0.1) was trained on a dataset where each RiseDecline row which included significant gaps in the lightcurves was removed. This is also in place when applied on real data. This will remove a lot of recurrent variability already at the design stage. 

Now we limit the sample to different number of detections (and other properties we might know). Each one of this cuts will produce a different classification model.

In [9]:
# Might want to redo this for different number of detections
# nbr_det = 2, 3, 4, 5, 6, 100
nbr_det = 2
if nbr_det<=2:
    max_duration = 3.5
    max_predetect = 3.5
    min_detmag = 16 # Things starting brighter than this must be stars
elif nbr_det<=4:
    max_duration = 6.5
    max_predetect = 3.5
    min_detmag = 16 # Things starting brighter than this must be stars
elif nbr_det<=6:
    max_duration = 10
    max_predetect = 3.5
    min_detmag = 16 # Things starting brighter than this must be stars
elif nbr_det==100:
    # This is a special case where we look for a full lightcurve
    max_duration = 90
    max_predetect = 10
    min_detmag = 16 # Things starting brighter than this must be stars    

Remove all lines for which the lightcurve has a gap. Important notes: 
1. These recurrent events might be most intresting to some. 
2. We just remove rows, so other copies of the same transient might still be there.

In [10]:
features = features.drop( np.where(features['bool_hasgaps'])[0] )

As the next step, also create a list of objects that were never detected again (beyond nbr_det). If possible we wish to train to get rid of these! Now there are many reasons we might not detect something, so there will again be good candidates here, but presumably not interesting for further follow-up. Kilnovae we anyway have to investigate using high cadence data.

In [11]:
iTime = np.where(features["t_lc"]<max_duration)[0]
features = features.iloc[iTime]

In [12]:
iPre = np.where(features["t_predetect"]<=max_predetect)[0]
features = features.iloc[iPre]

In [13]:
# Skip cases with intervening upper limits for now
if nbr_det<100:
    features = features[ features['bool_pure']]

In [14]:
# Also, lets skip the few cases with duplicated pps
iDup = np.where(features["cut_pp"]>0)[0]
dupTransients = features.iloc[iDup]['snname'].unique()
features = features[ ~features['snname'].isin(dupTransients) ]

Next, we look to keep transients that eventually get brighter than 18.5 (so into RCF territory)

In [15]:
iMag = np.where(features["mag_det"]<min_detmag)[0]
mTransients = features.iloc[iMag]['snname'].unique()
features = features[ ~features['snname'].isin(mTransients) ]

In [16]:
# Here we actually make a direct cut on each row
# Hmm, do we actually not do anything for 3 and five detections?   We actually run it for 2,3,4,5,6,6-100!
if nbr_det==100:
    iDet = np.where( (features["ndet"]<100) & (features["ndet"]>6) )[0]
else:
    iDet = np.where(features["ndet"]==nbr_det)[0]
features = features.iloc[iDet]

# Training

In [17]:
# We now create a classified sample where we remove everything with class 0
mlfeatures = features[ features['class']>0 ]

if nbr_det<100:
    # This is what is used for ndet<=6
    use_cols = ['mag_det','mag_last','t_lc','rb_med','col_det','t_predetect','distnr_med', 'magnr_med', 
                'classtar_med','sgscore1_med', 'distpsnr1_med', 'neargaia_med', 'maggaia_med']
else:
    # ANd for ndet=100
    use_cols = ['mag_det','mag_last','t_lc','rb_med','col_det','t_predetect','distnr_med', 
                 'magnr_med', 'classtar_med','sgscore1_med', 'distpsnr1_med',
                 'neargaia_med', 'maggaia_med','mag_peak','col_last','col_peak']

feats = mlfeatures[use_cols]
target = mlfeatures.rcf_sn

In [18]:
# Estimate scale_pos_weight to make up for class imbalance (negative_examples / positive_examples)
scale_pos_weight = (len(target) - np.sum(target)) / np.sum(target)

model = xgb.XGBClassifier(
    scale_pos_weight=scale_pos_weight,
    use_label_encoder=False,
    random_state=42,
    objective='binary:logistic')

In [19]:
param_grid = {
        'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        'min_child_weight': np.arange(0.0001, 0.5, 0.001),
        'gamma': np.arange(0.0,40.0,0.005),
        'learning_rate': np.arange(0.0005,0.5,0.0005),
        'subsample': np.arange(0.01,1.0,0.01),
        'colsample_bylevel': np.round(np.arange(0.1,1.0,0.01)),
        'colsample_bytree': np.arange(0.1,1.0,0.01),
        }

kfold = StratifiedKFold(
    n_splits=5, 
    shuffle=True, 
    random_state=42)

grid_search = RandomizedSearchCV(
    model, 
    param_grid, 
    scoring=None, 
#     n_iter = 200,                                # Maximum number of iterations
    n_iter = 1,
    n_jobs=4, 
    cv=kfold, 
    random_state=42, 
    verbose=1, 
    error_score='raise')

In [20]:
target

1531      False
1538      False
1596      False
1598      False
2012       True
          ...  
250126    False
250129    False
250130    False
250274    False
250298    False
Name: rcf_sn, Length: 2751, dtype: bool

In [21]:
grid_result = grid_search.fit(feats, target, eval_metric='aucpr')

Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [22]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_[ 'mean_test_score' ]
stds = grid_result.cv_results_[ 'std_test_score' ]
params = grid_result.cv_results_[ 'params' ]

Best: 0.878595 using {'subsample': 0.8, 'min_child_weight': 0.2681, 'max_depth': 3, 'learning_rate': 0.1675, 'gamma': 35.405, 'colsample_bytree': 0.3899999999999999, 'colsample_bylevel': 1.0}


In [23]:
best_estimator = grid_result.best_estimator_
best_estimator

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1.0,
              colsample_bynode=1, colsample_bytree=0.3899999999999999,
              enable_categorical=False, gamma=35.405, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.1675, max_delta_step=0, max_depth=3,
              min_child_weight=0.2681, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, predictor='auto',
              random_state=42, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=9.74609375, subsample=0.8, tree_method='exact',
              use_label_encoder=False, validate_parameters=1, verbosity=None)

In [24]:
print('Evaluating model on the whole training sample:')
pred = best_estimator.predict(feats)
precision = metrics.precision_score(target, pred)
recall = metrics.recall_score(target, pred)
aucpr = metrics.average_precision_score(target, pred)
print("Precision: %.2f%%" % (precision * 100.0))
print("Recall: %.2f%%" % (recall * 100.0))
print("AUCPR: %.2f%%" % (aucpr * 100.0))

Evaluating model on the whole training sample:
Precision: 46.78%
Recall: 93.75%
AUCPR: 44.44%


## A note on prediction values
When doing predictions with the estimator's `predict` or `predict_proba` methods, the return values are between 0.0 and 1.0, as expected.

In [25]:
best_estimator.predict_proba(feats)

array([[0.9700839 , 0.02991611],
       [0.3632251 , 0.6367749 ],
       [0.97369814, 0.02630189],
       ...,
       [0.97634256, 0.02365743],
       [0.59177387, 0.40822613],
       [0.81460595, 0.18539408]], dtype=float32)

However, when serializing the model for production it may happen that a plain floating point value is returned instead. We can simulate this by running the estimator's `predict` method with the `output_margin` argument.

In [26]:
margins = best_estimator.predict(feats, output_margin=True)
margins

array([-3.4789855 ,  0.5613934 , -3.6114604 , ..., -3.7201362 ,
       -0.37130323, -1.4802208 ], dtype=float32)

This should not be a big problem. In order to get a probablility-like value between 0.0 and 1.0 we just need to apply the logistic function to the margins.

In [27]:
1.0 / (1.0 + np.exp(-margins))

array([0.02991611, 0.63677484, 0.02630189, ..., 0.02365743, 0.40822613,
       0.18539408], dtype=float32)